In [21]:
from fitparse import FitFile
import matplotlib.pyplot as plt
import pandas as pd

fit_file = FitFile('fittest.fit')


In [86]:
power = []
heart_rate = []
time = []
cadence = []
for record in fit_file.get_messages("record"):
    # Records can contain multiple pieces of data (ex: timestamp, latitude, longitude, etc)
    for data in record:
        # Print the name and value of the data (and the units if it has any)
        if data.name == 'power':
            power.append(data.value)

        elif data.name == 'heart_rate':
            heart_rate.append(data.value)

        elif data.name == 'timestamp':
            time.append(data.value)

        elif data.name == 'cadence':
            cadence.append(data.value)

df = pd.DataFrame(list(zip(time, power, heart_rate, cadence)),
               columns =['timestamp', 'power', 'heart_rate', 'cadence'])

#df.head()


#get max/min
peak_p = max(power)
peak_h = max(heart_rate)
peak_cad = max(cadence)
min_p = min(power)
min_h = min(heart_rate)
min_cad = min(cadence)


min_time = df['timestamp'].min()
max_time = df['timestamp'].max()

#get length of session (seconds)
tot_time = (max_time - min_time).total_seconds()

# Find the minimum and maximum timestamps in your existing DataFrame
min_time = df['timestamp'].min()
max_time = df['timestamp'].max()

# Generate a new DataFrame with evenly spaced 1-second intervals
time_range = pd.date_range(start=min_time, end=max_time, freq='1S')
new_df = pd.DataFrame({'timestamp': time_range})

# Merge (join) the new DataFrame with your existing DataFrame based on the datetime column
merged_df = pd.merge(new_df, df, on='timestamp', how='left')

# Fill any missing values with appropriate defaults (e.g., NaN for power and heart_rate)
merged_df.fillna({'power': 0, 'heart_rate': 0}, inplace=True)
merged_df.tail()


,timestamp,power,heart_rate,cadence
3576,2023-09-28 13:23:06,52.0,127.0,51.0
3577,2023-09-28 13:23:07,49.0,126.0,33.0
3578,2023-09-28 13:23:08,45.0,126.0,15.0
3579,2023-09-28 13:23:09,0.0,125.0,0.0
3580,2023-09-28 13:23:10,0.0,125.0,0.0


In [87]:
# Calculate rolling averages for power and heart_rate at various time intervals
rolling_intervals = [5, 10, 15, 30, 60, 180, 300, 600, 1200, 2400, 3600, 5400, 7200, 10800]
result = {}

for interval in rolling_intervals:
    if interval <= tot_time:
        rolling_window = pd.Timedelta(seconds=interval)
        rolling_averages = []

        for i, row in merged_df.iterrows():
            end_time = row['timestamp']
            start_time = end_time - rolling_window
            data_within_window = merged_df[(merged_df['timestamp'] >= start_time) & (merged_df['timestamp'] <= end_time)]
            power_avg = data_within_window['power'].mean()
            heart_rate_avg = data_within_window['heart_rate'].mean()
            rolling_averages.append((start_time, end_time, power_avg, heart_rate_avg))

        result[f'power_rolling_{interval}s'] = rolling_averages

# Print the result
#print(result)
#for key, values in result.items():
 #   print(key)
  #  print(pd.DataFrame(values, columns=['Start Time', 'End Time', 'Power Avg', 'Heart Rate Avg']))

In [89]:
# Loop through the keys in the result dictionary
for key, values in result.items():
    max_power = max(values, key=lambda x: x[2])[2]  # Get the maximum power value
    max_heart_rate = max(values, key=lambda x: x[3])[3]  # Get the maximum heart rate value
    
    #print(f'Key: {key}')
    #print(f'Max Power: {max_power}')
    #print(f'Max Heart Rate: {max_heart_rate}')

In [71]:
tot_time

datetime.timedelta(seconds=3580)

In [ ]:
x = []
for i in range(len(heart_rate)):
  x.append(i)

plt.bar(x, heart_rate)
plt.show()